In [1]:
import torch 
from torch.utils.data import DataLoader
from src import full_pcd_dataset
import importlib
importlib.reload(full_pcd_dataset)
import json
# Create dataset instance
dataset = full_pcd_dataset.FullPCDDataset("data/full_pcd_30000_samples_6d.npz")
with open("data/label_dict.json", "r") as f:
    label_dict = json.load(f)
num_classes = len(label_dict.keys())
# Split into train and validation (example: 80-20 split)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4,pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4,pin_memory=True)
num_classes

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


5

In [2]:
from src.model_translation import TranslationModel 
from torch.optim.lr_scheduler import StepLR
model = TranslationModel()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = StepLR(optimizer, step_size=5, gamma=0.5) 

In [3]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [4]:
print(count_parameters(model))

276899


In [5]:
from src.train_position import train_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

2025-05-19 18:05:08.063920: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747667108.088312   53781 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747667108.093715   53781 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747667108.110294   53781 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747667108.110345   53781 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747667108.110349   53781 computation_placer.cc:177] computation placer alr

Using device: cuda


In [ ]:
train_model(model, train_loader, val_loader, optimizer, scheduler, device, epochs=30, directory="new_run_translation_no_norm_nobnl")

Training: 100%|██████████| 188/188 [09:25<00:00,  3.01s/it]


Epoch 1/30 - Total Loss: 0.7579


Validating: 100%|██████████| 47/47 [00:12<00:00,  3.80it/s]


Validation - Loss: 0.4953 | 


Training: 100%|██████████| 188/188 [09:35<00:00,  3.06s/it]


Epoch 2/30 - Total Loss: 0.5161


Validating: 100%|██████████| 47/47 [00:13<00:00,  3.59it/s]


Validation - Loss: 0.4980 | 


Training:   4%|▎         | 7/188 [00:21<09:22,  3.11s/it]